协程(coroutine)
- 实现并发编程的一种方式。多线程/多进程，是解决并发问题的经典模型之一。在最初的互联网世界，起到举足轻重的作用。

In [1]:
# 先看一个例子
import time


def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    time.sleep(sleep_time)
    print('OK {}'.format(url))


def main(urls):
    for url in urls:
        crawl_page(url)


%time main(['url_1', 'url_2', 'url_3', 'url_4'])


crawling url_1
OK url_1
crawling url_2
OK url_2
crawling url_3
OK url_3
crawling url_4
OK url_4
CPU times: total: 15.6 ms
Wall time: 10 s


In [8]:
# 引入协程
import asyncio


async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))


async def main(urls):
    for url in urls:
        await crawl_page(url)

await(main(['url_1', 'url_2', 'url_3', 'url_4']))


crawling url_1
OK url_1
crawling url_2
OK url_2
crawling url_3
OK url_3
crawling url_4
OK url_4


还是10秒,优化，引入任务(Task)

In [9]:
import asyncio


async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))


async def main(urls):
    tasks = [asyncio.create_task(crawl_page(url)) for url in urls]
    for task in tasks:
        await task

await(main(['url_1', 'url_2', 'url_3', 'url_4']))


crawling url_1
crawling url_2
crawling url_3
crawling url_4
OK url_1
OK url_2
OK url_3
OK url_4


执行tasks，另一种做法

In [10]:
import asyncio


async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))


async def main(urls):
    tasks = [asyncio.create_task(crawl_page(url)) for url in urls]
    await asyncio.gather(*tasks)

await(main(['url_1', 'url_2', 'url_3', 'url_4']))


crawling url_1
crawling url_2
crawling url_3
crawling url_4
OK url_1
OK url_2
OK url_3
OK url_4


解密协程运行时

In [12]:
import asyncio


async def worker_1():
    print('worker_1 start')
    await asyncio.sleep(1)
    print('worker_1 done')


async def worker_2():
    print('worker_2 start')
    await asyncio.sleep(2)
    print('worker_2 done')


async def main():
    print('before await')
    await worker_1()
    print('awaited worker_1')
    await worker_2()
    print('awaited worker_2')

await main()


before await
worker_1 start
worker_1 done
awaited worker_1
worker_2 start
worker_2 done
awaited worker_2


In [15]:
import asyncio


async def worker_1():
    print('worker_1 start')
    await asyncio.sleep(1)
    print('worker_1 done')


async def worker_2():
    print('worker_2 start')
    await asyncio.sleep(2)
    print('worker_2 done')


async def main():
    task1 = asyncio.create_task(worker_1())
    task2 = asyncio.create_task(worker_2())
    print('before await')
    await task1
    print('awaited worker_1')
    await task2
    print('awaited worker_2')

await main()


before await
worker_1 start
worker_2 start
worker_1 done
awaited worker_1
worker_2 done
awaited worker_2


In [1]:
import asyncio


async def worker_1():
    await asyncio.sleep(1)
    return 1


async def worker_2():
    await asyncio.sleep(2)
    return 2/0


async def worker_3():
    await asyncio.sleep(3)
    return 3


async def main():
    task_1 = asyncio.create_task(worker_1())
    task_2 = asyncio.create_task(worker_2())
    task_3 = asyncio.create_task(worker_3())

    await asyncio.sleep(2)
    task_3.cancel()

    res = await asyncio.gather(task_1, task_2, task_3, return_exceptions=True)
    print(res)

await main()


[1, ZeroDivisionError('division by zero'), CancelledError()]


协程实现一个生产者消费者模型

In [2]:
import random
import asyncio

async def consumer(queue,id):
    while True:
        val=await queue.get()
        print('{} get a val: {}'.format(id,val))
        await asyncio.sleep(1)
    
async def producer(queue,id):
    for _ in range(5):
        val = random.randint(1,10)
        await queue.put(val)
        print('{} put a val: {}'.format(id,val))
        await asyncio.sleep(1)
    
async def main():
    queue=asyncio.Queue()

    consumer_1=asyncio.create_task(consumer(queue,'consumer_1'))
    consumer_2=asyncio.create_task(consumer(queue,'consumer_2'))

    producer_1=asyncio.create_task(consumer(queue,'producer_1'))
    producer_2=asyncio.create_task(consumer(queue,'producer_2'))

    await asyncio.sleep(10)
    consumer_1.cancel()
    consumer_2.cancel()

    await asyncio.gather(consumer_1,consumer_2,producer_1,producer_2,return_exceptions=True)

await main()


    